We will be working with raw data from Open Stockholm and Open Weather Map APIs. Both APIs require API key for accessing data. Therefore, before working with tasks below, you need to obtain API keys for both data sources and understand how to work with these secrets securely.

API key for weather data
Create an account at Open Weather Map homepage and obtain API key under your account.

API key for Stockholm parking data
Obtain an API key from this link from Stockholm city- Open Stockholm homepage.

Working with secrets
When making a GET request to both APIs, you need to provide the API endpoint (URL) together with the API keys for authentication. However, you should not expose your API keys in your python scripts. Instead, you can make use of .env file. Make sure that this file IS NOT TRACKED by git. Check out this reference to understand how to use .env file to work with secrets.

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import json
import dlt
from datetime import datetime
import duckdb



# load_dotenv("openweatherAPI.env")

# API_KEY = os.getenv("API_KEY")
API_KEY = dlt.secrets.get("api_secret_key")
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

citys = ["Stockholm", "London", "Paris", "New York", "Tokyo"]





In [2]:
def fetch_weather_data(city):
    params = {
        "q":city, 
        "appid":API_KEY, 
        "units": "metric"
    ,}
    
    response = requests.get(BASE_URL, params=params)
    data = response.json() #json.loads(response.content.decode("utf8"))
    
    weather_dict = {
    "city": city,
    "timestamp": datetime.now().isoformat(),
    "temperature": data["main"]["temp"],
    "humidity": data["main"]["humidity"],
    "pressure": data["main"]["pressure"],
    "weather_description": data["weather"][0]["description"],
    "wind_speed": data["wind"]["speed"],
    "cloudiness": data["clouds"]["all"]
    }
    return weather_dict


@dlt.resource(write_disposition="replace")
def append_weather_data():
    for city in citys:
        yield fetch_weather_data(city)


def create_weather_pipeline():
    pipeline = dlt.pipeline(
        pipeline_name="weather_data_pipeline",
        destination=dlt.destinations.duckdb("weather_data.duckdb"),
        dataset_name="staging"
    )
    
    pipeline.run(append_weather_data(), table_name="weather_by_city")
    return pipeline


pipeline = create_weather_pipeline()

PipelineStepFailed: Pipeline execution failed at stage extract when processing package 1743686557.0390077 with exception:

<class 'dlt.extract.exceptions.ResourceExtractionError'>
In processing pipe append_weather_data: extraction of resource append_weather_data in generator append_weather_data caused an exception: 'main'

In [ ]:
with duckdb.connect("weather_data.duckdb") as con:
    result = con.execute("SELECT * FROM staging.weather_by_city").fetchall()
    for row in result:
        print(row)

('Stockholm', datetime.datetime(2025, 4, 3, 17, 8, 55, 786378, tzinfo=<DstTzInfo 'Europe/Stockholm' CEST+2:00:00 DST>), 19.47, 34, 1020, 'clear sky', 8.23, 0, '1743685735.6656818', 'FZa+Jo76WD+4Zg')
('London', datetime.datetime(2025, 4, 3, 17, 8, 55, 885646, tzinfo=<DstTzInfo 'Europe/Stockholm' CEST+2:00:00 DST>), 18.14, 48, 1019, 'few clouds', 4.47, 17, '1743685735.6656818', 'pDh1LU4/SjjL1A')
('Paris', datetime.datetime(2025, 4, 3, 17, 8, 55, 983717, tzinfo=<DstTzInfo 'Europe/Stockholm' CEST+2:00:00 DST>), 20.5, 42, 1017, 'clear sky', 7.2, 0, '1743685735.6656818', 'ddESZ9fcbm8xgQ')
('New York', datetime.datetime(2025, 4, 3, 17, 8, 56, 83787, tzinfo=<DstTzInfo 'Europe/Stockholm' CEST+2:00:00 DST>), 9.84, 93, 1019, 'mist', 2.24, 100, '1743685735.6656818', 'n+0kNUxrjXwqFQ')
('Tokyo', datetime.datetime(2025, 4, 3, 17, 8, 56, 183798, tzinfo=<DstTzInfo 'Europe/Stockholm' CEST+2:00:00 DST>), 7.86, 83, 1014, 'broken clouds', 5.14, 75, '1743685735.6656818', 'TlSq9tMGiVbfrA')


In [ ]:
pipeline.dataset().weather_by_city.df()

,city,timestamp,temperature,humidity,pressure,weather_description,wind_speed,cloudiness,_dlt_load_id,_dlt_id
0,Stockholm,2025-04-03 11:10:30.376138+00:00,13.76,54,1024,clear sky,5.14,0,1743671430.240295,OMr4wxx4GGClXg
1,London,2025-04-03 11:10:30.479370+00:00,13.94,64,1019,clear sky,6.17,2,1743671430.240295,CHoTozDvoM65FA
2,Paris,2025-04-03 11:10:30.597851+00:00,15.26,55,1017,clear sky,4.63,0,1743671430.240295,QqgY7mQxJLLUTA
3,New York,2025-04-03 11:10:30.704479+00:00,7.32,93,1022,overcast clouds,3.60,100,1743671430.240295,y/PtdTm5eUDF2A
4,Tokyo,2025-04-03 11:10:30.822041+00:00,8.48,83,1014,broken clouds,5.14,75,1743671430.240295,oAXikjqvTaOa0w
